# Understanding Prompt Templates

In [1]:
# Imports
import os
from langchain_openai import ChatOpenAI
from langchain.schema import AIMessage, SystemMessage, HumanMessage
from langchain import PromptTemplate
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, AIMessagePromptTemplate, HumanMessagePromptTemplate

api_key = os.getenv("OPENAI_API_KEY")

### With No Input Variables

In [2]:
# An example of Prompt with no input variables
no_input_prompt = PromptTemplate(input_variables=[], template="Tell me a fact")
no_input_prompt.format()

'Tell me a fact'

### With a Single Input Variable

In [3]:
one_input_prompt = PromptTemplate(input_variables=["topic"], template="Tell me a fact about {topic}")
one_input_prompt.format(topic="Mars")

'Tell me a fact about Mars'

### Multiple Input Variables

In [4]:
multiple_input_prompt = PromptTemplate(input_variables=["topic", "level"],
                                       template="Tell me about a fact about {topic} for a student at {level} level")
multiple_input_prompt.format(topic='Mars', level='8th grade')

'Tell me about a fact about Mars for a student at 8th grade level'

### Chat Model Templates

Chat models require a list of chat messages called a prompt, which is different from a raw string that you would input into a language model. Each message in the prompt is associated with a role, such as AI, human, or system.
For instance, when using the OpenAI Chat Completion API, a chat message can be assigned the role of AI, human, or system. The model is designed to pay closer attention to instructions provided in system chat messages.

To simplify the process of constructing and working with prompts, LangChain offers various prompt templates. It is highly recommended to utilize these chat-related prompt templates instead of PromptTemplate when interacting with chat models. This will allow you to fully harness the potential of the underlying chat model and enhance your experience.

We will favor these models in the course due to upcoming changes in the OpenAI ecosystem where chat agents will be favored over text completion models.

In [5]:
system_template = "You are an AI recipe Assistant that specialises in {dietary_preference} dishes that can prepared in {cooking_time}" 
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

# to check the input variables for the prompt template
system_message_prompt.input_variables

['cooking_time', 'dietary_preference']

In [6]:
human_template = "{recipe_request}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
human_message_prompt.input_variables

['recipe_request']

In [7]:
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
chat_prompt.input_variables

['cooking_time', 'dietary_preference', 'recipe_request']

In [8]:
# Now we can get a chat completion from the formatted messages
chat_prompt.format_prompt(cooking_time='15 min', dietary_preference='Non-Veg', recipe_request='chicken salad').to_messages()

[SystemMessage(content='You are an AI recipe Assistant that specialises in Non-Veg dishes that can prepared in 15 min', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='chicken salad', additional_kwargs={}, response_metadata={})]

#### Now we make the LLM Call

In [9]:
chat = ChatOpenAI(openai_api_key = api_key)

request = chat_prompt.format_prompt(cooking_time='15 min', dietary_preference='Non-Veg', recipe_request='chicken salad').to_messages()

result = chat(request)

/var/folders/ny/r3wsv_bd0bs1_vcy4pb5q9pr0000gn/T/ipykernel_93450/1560911232.py:5: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = chat(request)


In [10]:
print(result.content)

Here is a quick and easy recipe for a delicious Chicken Salad that can be prepared in 15 minutes:

Ingredients:
- 2 cups cooked and shredded chicken
- 1/2 cup mayonnaise
- 1/4 cup chopped celery
- 1/4 cup chopped red onion
- 1/4 cup chopped pickles
- 1 tablespoon Dijon mustard
- Salt and pepper to taste

Instructions:
1. In a large mixing bowl, combine the shredded chicken, mayonnaise, celery, red onion, pickles, and Dijon mustard.
2. Mix well until all the ingredients are evenly coated with the mayonnaise and mustard.
3. Season with salt and pepper to taste, and adjust seasoning as needed.
4. Serve the chicken salad on its own, on a bed of lettuce, or in a sandwich or wrap.
5. Enjoy your delicious and quick Chicken Salad!

Feel free to customize this recipe by adding ingredients like grapes, nuts, or herbs to suit your taste preferences. Enjoy!


#### One More Example - All Encapsulated within a function

In [11]:
def travel_idea (interest, duration):
    '''
    interest : destination where you want to go
    durartion : no of days you want your itinerary to be
    '''

    # SYSTEM
    system_template = "You are an AI Travel Agent that helps people to arrange detailed itinerary for the destination {interest} for the {duration} days"
    system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
    # HUMAN REQUEST
    human_template = "{human_request}"
    human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
    # COMPILE TO CHAT
    chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
    # INSERT VARIABLES
    request = chat_prompt.format_prompt(interest=interest, duration=duration, human_request="Please give a detailed day wise vacation itinerary for")
    # CHAT REQUEST
    chat = ChatOpenAI(openai_api_key=api_key)
    result = chat(request)
    return result.content

In [12]:
print(travel_idea('Manali', 5))

Day 1:
- Arrival in Manali
- Check-in at your hotel and freshen up
- Visit Hadimba Devi Temple, a beautiful wooden temple surrounded by cedar forests
- Explore the Manali Market for shopping and local cuisine
- Relax and unwind at the hotel in the evening

Day 2:
- Breakfast at the hotel
- Proceed to Solang Valley for adventure activities like paragliding, zorbing, and ropeway ride
- Enjoy lunch at a local restaurant in Solang Valley
- Visit the Rohtang Pass (subject to availability, as it may be closed in certain seasons) for breathtaking views and snow activities
- Return to the hotel for dinner and overnight stay

Day 3:
- Breakfast at the hotel
- Drive to Kullu to explore the picturesque Kullu Valley
- Visit the Naggar Castle, an ancient castle with scenic views
- Explore the Shawl Factory and shop for traditional Kullu shawls
- Enjoy river rafting in the Beas River
- Return to Manali in the evening for dinner and relaxation

Day 4:
- Breakfast at the hotel
- Visit the Vashisht Tem

In [13]:
print(travel_idea('Turkey', 9))

Day 1: Arrival in Istanbul
- Arrive in Istanbul and check into your hotel
- Visit the historic Sultanahmet Square, home to iconic landmarks like the Blue Mosque and Hagia Sophia
- Explore the Grand Bazaar for shopping and local cuisine
- Enjoy a traditional Turkish dinner at a local restaurant

Day 2: Istanbul
- Visit Topkapi Palace, the former royal residence of the Ottoman Sultans
- Explore the bustling Taksim Square and Istiklal Street for shopping and dining
- Take a Bosphorus cruise to see the city from the water
- Visit the Spice Bazaar for a sensory experience of Turkish spices and sweets

Day 3: Istanbul
- Explore the ancient underground Basilica Cistern
- Visit the Chora Church to see stunning Byzantine mosaics
- Relax at a traditional Turkish hammam for a spa experience
- Enjoy a dinner cruise on the Bosphorus with live entertainment

Day 4: Cappadocia
- Fly to Cappadocia and check into your cave hotel
- Visit the Goreme Open-Air Museum to see ancient cave churches
- Explore 